"""
Author: 
Micah vandersteen

Date:
25 January 2020
"""

In [ ]:
# IMPORTING DEPENDENCIES 
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
from pprint import pprint
import os

# IMPORT API KEY
from config import g_key

Store Part I results into DataFrame
--> Load the csv exported in Part I to a DataFrame

In [ ]:
# READING IN RESULTS FROM WEATHERPY SCRIPT OUTPUTFILE FROM THE OUTPUT DATA FOLDER
weather_df = pd.read_csv("../output_data/weather.csv")
weather_df.head()

Create a Humidity Heatmap using the found cities and their humidities.

In [ ]:
# CONFIGURE GMAPS WITH API KEY
gmaps.configure(api_key = g_key)

# STORING LATITUDE AND LONGITUDE INTO LOCATIONS LIST
locations = weather_df[["Latitude", "Longitude"]].astype(float)
humidity = weather_df["Humidity (%)"].astype(float)

# CREATING HEATMAP OF FOUND CITIES' HUMIDITY PERCENTAGES
fig = gmaps.Map()

heatmap_layer = gmaps.heatmap_layer(locations,
                                    weights = humidity, 
                                    dissipating = False,
                                    max_intensity = 100,
                                    point_radius = 2)

fig.add_layer(heatmap_layer)

fig



Create new DataFrame fitting weather criteria:

A max temperature lower than 90 degrees but higher than 75.
Wind speed less than 5 mph.
Less than 25 % cloudiness.
Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.


In [ ]:
# LOCATING ROWS THAT FIT ALL THREE SPECIFICATIONS DEFINED ABOVE
ideal_cities_df = weather_df.loc[(weather_df["Cloudiness (%)"] < 25) & (weather_df["Wind Speed (MPH)"] < 5) & (weather_df["Temperature (F)"] > 75) & (weather_df["Temperature (F)"] < 90)]


In [ ]:
# showing ideal cities dataframe
ideal_cities_df

In [ ]:
# USE GOOGLE PLACES API TO FIND NEAREST HOTEL (WITHIN 5000 METERS) TO EACH IDEAL WEATHER CITY FROM 'ideal_cities_df'
# INITIALIZING A LIST OF MY IDEAL CITIES COORDINATES
ideal_cities_longitudes = list(ideal_cities_df["Longitude"])

ideal_cities_latitudes = list(ideal_cities_df["Latitude"])

# DEFINING WHAT TO SEARCH FOR, SEARCH RADIUS IN METERS
search_for = "lodging"

search_radius = "5000"

# ITERATION VARIABLE 
i = 0 

# DEFINING DESIRED VARIABLE LISTS TO GET FROM API INFORMATION
hotel_names = []

# FOR LOOP TO QUERY NEAREST HOTEL IN EACH IDEAL CITY
for i in np.arange(len(ideal_cities_df["Longitude"])):
    
    
    # BASE URL FOR GOOGLE PLACE API CALL
    base_url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={ideal_cities_latitudes[i]},\
            {ideal_cities_longitudes[i]}&radius={search_radius}&type={search_for}&key={g_key}"
    
    # GET RESPONSE AND JSONIFY
    response = requests.get(base_url).json()
    
    # APPENDING APPROPRIATE RESPONSES TO CORRESPONDING LISTS
    hotel_names.append(response['results'][0]['name'])
    
    i+=1

In [ ]:
# ADDING HOTEL NAME COLUMN TO IDEAL CITIES DATAFRAME USING THE FOUND HOTELS IN 'hotel_names' list
ideal_cities_df["Hotel Name"] = hotel_names

In [ ]:
# SHOWING NEW DATAFRAME
ideal_cities_df

In [ ]:
# ADD HOTEL NAMES TO THE LOCATIONS ON ORIGINAL HEATMAP
# ASSIGNING MARKER VARIABLE
coordinates = ideal_cities_df[["Latitude", "Longitude"]].astype(float)
city_names = list(ideal_cities_df["City Name"])

i = 0 

info_boxes = []

for i in np.arange(len(hotel_names)):
    
    info_boxes.append(f"""
    HOTEL NAME:
    {hotel_names[i]}
    
    CITY NAME:
    {city_names[i]}
    """)
    
    i+=1
    
markers = gmaps.marker_layer(coordinates,
                            info_box_content = info_boxes)
# Add the layer to the map
fig.add_layer(markers)

fig